In [16]:
!pip install pandas datasets transformers evaluate numpy


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [17]:


import pandas as pd

model_checkpoint = "distilbert-base-uncased"
batch_size = 16

# Replace the data paths
df_train = pd.read_csv("/kaggle/input/parspec/train_data.csv")
df_train =df_train.dropna(subset=['text'])
df_test = pd.read_csv("/kaggle/input/parspec/test_data.csv")
df_test =df_test.dropna(subset=['text'])

mapping_dict = {j:i for i,j in enumerate(df_train.target_col.unique())}

import re

def text_clean(x):
    """
    Cleans text by applying light preprocessing steps:
    - Converts to lowercase
    - Removes unicode characters
    - Removes URLs
    - Removes contractions
    - Removes words with digits
    - Removes excess whitespace
    """
    try:
      x = x.lower()  # Convert to lowercase
      x = x.encode('ascii', 'ignore').decode()  # Remove non-ASCII characters
      x = re.sub(r'http[s]?://\S+', ' ', x)  # Remove URLs
      x = re.sub(r'www?\S+', ' ', x)  # Remove URLs
      x = re.sub(r"'\w+", '', x)  # Remove contractions (optional)
      # x = re.sub(r'\w*\d+\w*', '', x)  # Remove words with numbers
      x = re.sub(r'\s{2,}', ' ', x)  # Reduce multiple spaces to one
      x = re.sub(r'^\s+|\s+$', '', x)  # Trim leading/trailing spaces
      x = re.sub(r'\n|\r', ' ', x)  # Remove newlines
    except:
      print(x)
    return x

def clean_df(df_train):
  df_train =df_train.dropna(subset=['text'])
  df_train['text_cleaned'] = df_train['text'].apply(lambda x: text_clean(x))
  df_train['label'] = df_train['target_col'].apply(lambda x: mapping_dict.get(x))
  return df_train[['text_cleaned','label']]

df_train = clean_df(df_train)
df_test = clean_df(df_test)


## Training the model


In [18]:


from datasets import Dataset

# Assuming 'df' is your pandas DataFrame
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)
# Now you can work with the 'dataset' object, which is a Hugging Face dataset
# Example:

label2id = mapping_dict
id2label = {v:k for k,v in mapping_dict.items()}

df_test.label.value_counts()

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text_cleaned"], padding="max_length", truncation=True)


tokenized_dataset_train = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset_test = test_dataset.map(tokenize_function, batched=True)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased",
                                                           num_labels=4,id2label=id2label,
                                                           label2id=label2id).to("cuda")

# freeze base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

import evaluate

accuracy = evaluate.load("accuracy")
import numpy as np
def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc}





/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
lr = 2e-4
batch_size = 8
num_epochs = 10
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="bert-parspec-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    load_best_model_at_end=True,
)

In [20]:


from transformers import Trainer,DataCollatorWithPadding
data_collator =DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,1.100000,0.944668,0.694000
2,0.658000,0.597263,0.763000
3,0.428200,0.474491,0.849000
4,0.327500,0.439122,0.861000
5,0.265700,0.423168,0.853000
6,0.235900,0.370328,0.873000
7,0.199300,0.371577,0.873000
8,0.192600,0.333994,0.882000
9,0.178900,0.363674,0.878000
10,0.187700,0.359447,0.878000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=970, training_loss=0.3773748220856657, metrics={'train_runtime': 404.4008, 'train_samples_per_second': 38.205, 'train_steps_per_second': 2.399, 'total_flos': 4065138802483200.0, 'train_loss': 0.3773748220856657, 'epoch': 10.0})

In [ ]:
trainer.push_to_hub("saurabhgssingh/parspec",token="<fill-here")

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/saurabhgssingh/bert-phishing-classifier_teacher/commit/6a98189dd37e6d54844e1aa2956e94b1cf0ad850', commit_message='saurabhgssingh/parspec', commit_description='', oid='6a98189dd37e6d54844e1aa2956e94b1cf0ad850', pr_url=None, repo_url=RepoUrl('https://huggingface.co/saurabhgssingh/bert-phishing-classifier_teacher', endpoint='https://huggingface.co', repo_type='model', repo_id='saurabhgssingh/bert-phishing-classifier_teacher'), pr_revision=None, pr_num=None)

# Evaluating the prediction on test dataset

In [23]:
import evaluate
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

accuracy = evaluate.load("accuracy")
import numpy as np
def compute_test_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
    precision = precision_score(labels, predicted_classes, average='macro')
    recall = recall_score(labels, predicted_classes, average='macro')
    f1 = f1_score(labels, predicted_classes, average='macro')
    report = classification_report(labels, predicted_classes)
    print("Precision (macro):", precision)
    print("Recall (macro):", recall)
    print("F1 Score (macro):", f1)
    print("Classification Report:\n", report)
    print("Accuracy:\n", acc)



In [24]:
predictions = trainer.predict(tokenized_dataset_test)

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
compute_test_metrics((logits, labels))


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Precision (macro): 0.8709337705305447
Recall (macro): 0.812882771273477
F1 Score (macro): 0.811384470706227
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96        78
           1       0.88      0.99      0.93        83
           2       0.86      0.34      0.49        35
           3       0.78      0.96      0.86        49

    accuracy                           0.88       245
   macro avg       0.87      0.81      0.81       245
weighted avg       0.88      0.88      0.86       245

Accuracy:
 0.882


# Inference

In [2]:


import pandas as pd

model_checkpoint = "distilbert-base-uncased"
batch_size = 16

df_train = pd.read_csv("/kaggle/input/parspec/train_data.csv")
df_train =df_train.dropna(subset=['text'])
df_test = pd.read_csv("/kaggle/input/parspec/test_data.csv")
df_test =df_test.dropna(subset=['text'])

In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="saurabhgssingh/bert-phishing-classifier_teacher",top_k=None, padding="max_length", truncation=True)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [11]:
pipe.predict(df_test.text_cleaned[2])[0]

[{'label': 'lighting', 'score': 0.967765748500824},
 {'label': 'others', 'score': 0.028922146186232567},
 {'label': 'cable', 'score': 0.0018619885668158531},
 {'label': 'fuses', 'score': 0.0014499942772090435}]